## Permissões POO para Python / Conexão com SQLite:

In [ ]:
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

# Dentro do pacote ORM existe o método 'declarativeBase' que permite que o Python crie as classes que irão
# representar as tabelas do banco de dados relacional.
# Toda classe que herdar de Base vai virar uma tabela no banco.”
# A classe pai 'base' será herdada por todas as entidades.


from sqlalchemy import create_engine

DATABASE_URL = "sqlite:///biblioteca.db"

# sqlite → dialeto do banco
# /// → banco local em arquivo
# biblioteca.db → arquivo do banco
# Isso cria um arquivo biblioteca.db na raiz do projeto (se ainda não existir).

engine = create_engine(DATABASE_URL, echo=True)

# A 'engine' é o motor de um carro. Ela que irá fazer o carro mover e se locomover. Sendo assim, 
# a 'engine' que irá levar os comandos Python até a linguagem final SQL
# A 'engine' gerencia as conexões, enviar os comandos Python, transformam em comandos SQL e recebe resutados

# O 'echo=True' irá mandar todos os comandos feitos em Python, transformados em SQL, mostrar na tela do terminal.
# Com o 'echo=False', será criado o banco sem mostrar os comandos SQL.

# Base ──► define modelos (classes)
#   │
# engine ──► conecta ao banco (Sqlite)
#   │
# SessionLocal ──► cria sessões
#   │
# session ──► lê / grava dados

# Resumo: Base define o modelo, a engine conecta ao banco e a Session é quem trabalha com os dados.

# Atenção!

# O DuckDB NÃO suporta o tipo SERIAL. = (Integer + PK + Auto_increment)
# Catalog Error: Type with name SERIAL does not exist!
# Em bancos como PostgreSQL, isso vira SERIAL. No DuckDB, isso quebra, porque SERIAL não existe.
# Para isso, deverá colocar 'Identity column (autoincrement compatível)'

# Notas importantes sobre o banco de dados DuckDB:

# O DuckDB é:

# excelente para analytics e excelente para OLAP.
# Mas não focado em constraints relacionais complexas por isso :

# FK → funciona parcialmente
# UNIQUE → não implementado
# CHECK → limitado

# Para não dar errado na execução do projeto, não coloque UNIQUES nos atributos.

# DuckDB não é banco relacional clássico:
# ótimo para análise
# ruim para constraints

# Para um projeto de biblioteca (OLTP):
# PostgreSQL → ideal
# SQLite → ok para estudo
# DuckDB → só se o foco for analytics

## Entidade Usuário:

In [2]:
from sqlalchemy import Column, Integer, VARCHAR, CHAR, ForeignKey
from sqlalchemy.orm import relationship

class Usuario(Base):
    __tablename__ =  "usuario"
    id_usuario = Column(Integer, primary_key=True)
    nome = Column(VARCHAR(50), nullable=False)
    matricula = Column(CHAR(10),unique=True)
    email = Column(VARCHAR(20), nullable=False)
    telefone = Column(VARCHAR(15), nullable=False)
    endereco = Column(VARCHAR(50), nullable=False)

    # Relação: A entidade 'usuario' tem relação direta com a entidade 'emprestimo':
    emprestimos = relationship("Emprestimo", back_populates="usuario")

    __mapper_args__= {"polymorphic_identity": "usuario", "polymorphic_on": None}
# Especialização:
class Aluno(Usuario):
    __tablename__= "aluno"
    id_usuario = Column(Integer, ForeignKey("usuario.id_usuario"), primary_key=True)  
    __mapper_args__={"polymorphic_identity":"aluno",} 
# Especialização:
class Professor(Usuario):
    __tablename__= "professor"
    id_usuario=Column(Integer, ForeignKey("usuario.id_usuario"),primary_key=True)    

    __mapper_args__={"polymorphic_identity":"professor",}          



## Entidade Livros:

In [3]:
from sqlalchemy import Column, Integer, Date, VARCHAR
from sqlalchemy.orm import relationship

class Livro(Base):
    __tablename__ = "livro"
    id_livro = Column(Integer, primary_key=True)
    titulo = Column(VARCHAR(50), nullable=False)
    editora = Column(VARCHAR(50))
    ano_publicacao = Column(Date)
    quantidade_estoque = Column(Integer, nullable=False)

    # Relação: A entidade 'livro' se relaciona diretamente com a entidade 'item_emprestimo'

    itens = relationship("ItemEmprestimo", back_populates="livro")
    


## Entidade item_emprestimo:

In [4]:
from sqlalchemy import Column, Integer, DateTime, ForeignKey, UniqueConstraint
from sqlalchemy.orm import relationship

class ItemEmprestimo(Base):
    __tablename__ = "item_emprestimo"
    id_emprestimo = Column(Integer, ForeignKey("emprestimo.id_emprestimo"), primary_key=True)
    id_livro = Column(Integer, ForeignKey("livro.id_livro"), primary_key=True)
    data_prev_entrega = Column(DateTime, nullable=False)
    data_devolucao = Column(DateTime, nullable=False)

    # Relações: A entidade 'item_emprestimo' tem relação direta com as entidades 'livro' e 'empréstimo'

    emprestimo = relationship("Emprestimo", back_populates="itens")    
    livros = relationship("Livro", back_populates="itens")

    __table_args__ = (UniqueConstraint(
        "id_emprestimo", "id_livro", name = "uq_item_emprestimo"),)
    


## Entidade funcionario:

In [5]:
from sqlalchemy import Column, Integer, VARCHAR, CHAR
from sqlalchemy.orm import relationship

class Funcionario(Base):
    __tablename__ = "funcionario"
    id_funcionario = Column(Integer,primary_key=True)
    nome = Column(CHAR(50),nullable=False)
    cpf= Column(VARCHAR(15),unique=True, nullable=False)
    telefone = Column(VARCHAR(50),unique=True,nullable=False)
    email = Column(VARCHAR(50),nullable=False)
    cargo = Column(CHAR(15),nullable=False)

    # Relacionamentos: A entidade 'funcionario' se relaciona com 'empréstimo' diretamente:

    emprestimo =relationship ("Emprestimo", back_populates= "funcionario")

## Entidade Empréstimo:

In [6]:

from sqlalchemy import Column, Integer, ForeignKey, DateTime
from sqlalchemy.orm import relationship
from datetime import datetime, timezone

# Representa o ato administrativo do empréstimo:
class Emprestimo(Base):
    __tablename__ = "emprestimo"

    id_emprestimo = Column(Integer, primary_key=True, autoincrement=True)
    id_usuario = Column(Integer, ForeignKey("usuario.id_usuario"), nullable=False)
    id_funcionario = Column(Integer, ForeignKey("funcionario.id_funcionario"), nullable=False)
    data_emprestimo = Column(DateTime, default=lambda: datetime.now(timezone.utc))

    # Relacionamentos: A entidade se relaciona diretamente com 'item_emprestimo','funcionario','usuario':
    funcionario = relationship("Funcionario", back_populates="emprestimo")
    usuario = relationship("Usuario", back_populates="emprestimo")
    itens = relationship(
        "ItemEmprestimo",
        back_populates="emprestimo",
        cascade="all, delete-orphan"
    )


## Criação das tabelas do banco:

In [7]:
Base.metadata.create_all(bind=engine)

2026-01-03 22:11:02,377 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-03 22:11:02,379 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("usuario")
2026-01-03 22:11:02,383 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-01-03 22:11:02,388 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("usuario")
2026-01-03 22:11:02,391 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-01-03 22:11:02,394 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("aluno")
2026-01-03 22:11:02,396 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-01-03 22:11:02,399 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("aluno")
2026-01-03 22:11:02,402 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-01-03 22:11:02,405 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("professor")
2026-01-03 22:11:02,406 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-01-03 22:11:02,412 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("professor")
2026-01-03 22:11:02,413 INFO sqlalchemy.engine.Engine [raw sql] ()